In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import holoviews as hv
hv.extension('bokeh')
from holoviews.operation.datashader import datashade

import stochastic_sc_model as ssc_model

# Ensure we don't hit floating point errors
np.seterr('raise')

The problem is defined by the location of the currently active polymerases. Each polymerase is characterized by:
1. It's location
2. The length of the nascent mRNA tail ($x$)
3. The DNA twist at this location ($\phi$)
4. The RNAC at this location ($\theta$)

We actually only need to track the first three; the fourth is completely determined by the first three.

Given this state information, we can compute the relaxed supercoiling, which in turn gives us information on the 

In [ ]:
params = {
    'mRNA_drag': 1/20, # pN nm^(alpha / 1)
    'mRNA_exponent': 1, # the value of alpha
    'DNA_twist_mobility': 10, # s pN nm
    'RNAP_radius': 15, # nm
    'RNAP_velocity': 20, # nm / s
    'RNAP_torque_cutoff': 12, # pN nm
    'RNAP_stall_torque_width': 3, #pN
    'DNA_force': 1, # pN
    'DNA_bend_plength': 50, # pN
    'DNA_twist_plength': 95, # pN
    'DNA_plectonome_twist_plength': 24, # pN
    'temperature': 298, # K
    'base_promoter_initiation_rate': 1 / 120, # 1 / sec
    'topo_rate': 1 / 1200, # 1 / sec
    'mRNA_deg_rate': 1 / 1200 # 1 / sec
}

In [ ]:
result = ssc_model.bulk_simulation(params, ('free', (10000, 0)), [(1000, 700, 1), (1700, 1300, 1)],
                ['upstream', 'downstream'], (0, 10000, 1000), 100)

In [ ]:
combo_plots = hv.Scatter(result, 'time', 'upstream') * hv.Scatter(result, 'time', 'downstream')
datashade(combo_plots)

In [ ]:
sim = SupercoilingSimulation(params, ('free', (10000, 0)), [(1000, 700, 1),  (1700, 1300, 1)])
sim.enable_topo_relaxation()

In [ ]:
postprocess['gene_expression'][:,1]

In [ ]:
np.random.seed(1)
example_run = sim.simulate((0,10000))
for segment in example_run[0]:
    plt.plot(segment[0], segment[1][::3,:].T, 'c')

In [ ]:
postprocess = sim.postprocess_run(example_run)

In [ ]:
print(postprocess['time'].shape)
print(postprocess['x_domain'].shape)
print(postprocess['excess_twist'].shape)
print(postprocess['sc_density'].shape)
postprocess['gene_expression'].shape

In [ ]:
plt.plot(postprocess['time'], postprocess['gene_expression'])
for topo_event in example_run[1]['topo']:
    plt.axvline(topo_event)
plt.show()
plt.imshow(postprocess['excess_twist'].T)
plt.colorbar()
plt.show()
plt.imshow(postprocess['sc_density'].T)
plt.colorbar()
plt.show()

In [ ]:
plt.plot(np.linspace(0,.05,100),sim.model.polymerase_velocity(np.zeros((100,)), np.linspace(0,.05,100)))
plt.show()
plt.plot(np.linspace(0,20,100), 1 / (1 + np.exp((np.linspace(0,20,100) - 12)/.1)))
plt.show()
plt.plot(np.linspace(0,.1,100), sim.model.torque_response(np.linspace(0,.1,100)))